In [1]:
!pip install socket

ERROR: Could not find a version that satisfies the requirement socket (from versions: none)
ERROR: No matching distribution found for socket


In [2]:
!pip install geopy

In [3]:
!pip install folium


In [4]:
!pip install pyproj

In [5]:
!pip install seaborn

In [6]:
!pip install matplotlib

In [7]:
import socket
import geopy
import folium
import pyproj
import seaborn as sns
import matplotlib.pyplot as plt
import ipaddress
import subprocess
import concurrent.futures
from datetime import datetime
import platform

In [8]:
def get_local_ip():
    import socket
    s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    s.connect(("8.8.8.8",80))
    local_ip=s.getsockname()[0]
    s.close()
    return local_ip




In [9]:
from ast import Param
def ping_ip(ip):
    """Ping an IP address to check if it's active"""
    param = '-n' if platform.system().lower() == 'windows' else '-c'
    command = ['ping', param, '1', str(ip)]
    try:
        output = subprocess.check_output(command, stderr=subprocess.STDOUT, universal_newlines=True)
        return ip if '1 received' in output or 'bytes from' in output else None
    except subprocess.CalledProcessError:
        return None



In [10]:
def scan_local_network():
    # Get local IP
    local_ip = get_local_ip()
    if not local_ip:
        print("Could not determine local IP address")
        return []

    # Create network address
    ip_parts = local_ip.split('.')
    network = f"{ip_parts[0]}.{ip_parts[1]}.{ip_parts[2]}.0/24"

    print(f"Local IP: {local_ip}")
    print(f"Scanning network: {network}")
    print("This may take a few minutes...\n")

    start_time = datetime.now()
    active_ips = []

    try:
        # Use ThreadPoolExecutor for parallel scanning
        with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
            # Create list of all possible IPs in the network
            ip_list = list(ipaddress.IPv4Network(network).hosts())
            # Create futures for each IP
            future_to_ip = {executor.submit(ping_ip, ip): ip for ip in ip_list}

            # Process completed futures
            for future in concurrent.futures.as_completed(future_to_ip):
                result = future.result()
                if result:
                    active_ips.append(str(result))

        scan_duration = datetime.now() - start_time
        print(f"\nScan completed in {scan_duration.total_seconds():.2f} seconds")
        print(f"Found {len(active_ips)} active IPs")

    except Exception as e:
        print(f"Error during scan: {e}")

    return active_ips

# Usage:
if __name__ == "__main__":
    active_ips = scan_local_network()
    # Print results
    for ip in sorted(active_ips):
        try:
            hostname = socket.gethostbyaddr(ip)[0]
            print(f"IP: {ip:<15} Hostname: {hostname}")
        except socket.herror:
            print(f"IP: {ip:<15} Hostname: Unknown")

Local IP: 172.28.0.12
Scanning network: 172.28.0.0/24
This may take a few minutes...

Error during scan: [Errno 2] No such file or directory: 'ping'


In [20]:
def location_coordinates():
    for ip in sorted(active_ips):
        latitude = 33.0975
        longitude = -97.3080
        print(f"IP: {ip:<15} Latitude: {latitude:<10} Longitude: {longitude}")

# Usage:
if __name__ == "__main__":
    active_ips = scan_local_network()
    # Print results with hostnames
    for ip in sorted(active_ips):
        try:
            hostname = socket.gethostbyaddr(ip)[0]
            print(f"IP: {ip:<15} Hostname: {hostname}")
        except socket.herror:
            print(f"IP: {ip:<15} Hostname: Unknown")

    # Call location_coordinates() after the scan
    location_coordinates()

Local IP: 172.28.0.12
Scanning network: 172.28.0.0/24
This may take a few minutes...

Error during scan: [Errno 2] No such file or directory: 'ping'


-Getting the IP address for a specific location is not possible, mabe with paid third party access it could be


RESULT; Ones' private ip address is displayed indicating controlled packet access

TRIAL; Ping exact location coordinates in wireshark/network monitoring tools and monitor packets based in enhanced third party access